# Tuto : Comment créer une IA qui dit si les choses sont de gauche ou de droite ?

Dans ce notebook python, je vais expliquer comment construire une intelligence artificielle (IA) qui vous dit si les choses sont de gauche ou de droite. Ce que je vais expliquer ici, c'est ce que j'ai fait pour le site https://degaucheoudedroite.delemazure.fr. Si vous voulez des détails sur le fonctionnement de cette IA, sur l'histoire derrière le site, et sur les statistiques des recherches des utilisateurs, je vous invite à lire les articles sur mon blog. Dans [la première partie](https://theo.delemazure.fr/blog/degaucheoudedroite/histoire.html), je raconte l'histoire du site et je tente d'expliquer le fonctionnement de l'intelligence artificiel. Dans [la seconde partie](https://theo.delemazure.fr/blog/degaucheoudedroite/statistiques.html), j'analyse les statistiques des recherches faites sur le site, et les résultats sont assez intéressants.

Ici, je vais donc expliquer la méthode que j'ai utilisée pour classifier les mots à droite ou à gauche, qui va vous étonner par sa simplicité. Si vous venez chercher une technique révolutionnaire de NLP (traitement automatique des langues), vous allez être déçus. ;)

Pour construire cette IA, j'ai donc utilisé [GPT-3 de OpenAI](https://fr.wikipedia.org/wiki/GPT-3), une intelligence artificielle de complétion de texte basée sur un réseau Transformers. Je ne vais pas revenir en détail sur son fonctionnement, si vous voulez en savoir plus vous pouvez toujours [lire mon article](https://theo.delemazure.fr/blog/degaucheoudedroite/histoire.html) ou bien [cet article en anglais ](https://medium.com/sciforce/what-is-gpt-3-how-does-it-work-and-what-does-it-actually-do-9f721d69e5c1) qui explique bien son fonctionnement. Pour résumer de façon très très vulgarisée, l'IA est entrainée sur une grosse partie d'internet à lire des textes et à les compléter. On indique à l'IA si elle s'est trompée ou si elle est proche de la réalité, afin que celle-ci se perfectionne pour les complétions suivantes. 

L'avantage, c'est qu'il n'y a pas besoin de savoir comment ça marche précisément (même si c'est toujours bien) pour pouvoir l'utiliser. Par contre, il faut savoir comment interagir avec cette IA : quel type de texte faut-il lui donner à compléter ? Comment choisir les paramètres ? Ces questions peuvent paraître anodines, mais elles sont cruciales. Comme ce n'est pas la première fois que j'utilise cette IA pour faire des petits projets de la sorte, je commence à bien maîtriser cette IA et à rapidement obtenir les résultats désirés.

Bon. Venons en aux faits. Comment faire une IA comme "de gauche ou de droite" sans connaissances en NLP ?

D'abord, il faut [se créer un compte sur OpenAI](https://beta.openai.com/) pour pouvoir utiliser GPT-3. Le site va ensuite vous donner un identifiant propre à votre compte (*organization key*) et une clé secrète (*secret key*).


In [1]:
ORGANIZATION = 
KEY = 

On utilise alors l'une des librairies disponibles pour faire des requêtes à GPT-3. Par exemple, en python, il suffit d'installer et d'importer la librairie **openai**.

In [2]:
import openai
openai.organization = ORGANIZATION
openai.api_key = KEY

Ensuite, on construit une mini base de données d'entraînement. Pas besoin d'énormément de données, l'idée est juste que l'IA comprenne la tâche qui lui est demandée. Par exemple, dans mon cas, je veux juste faire passer le message qu'elle doit me répondre uniquement par *"De gauche"*, *"De droite"* ou *"Les deux"*. Les exemples servent aussi à calibrer : *Zemmour* c'est de droite, *Mélenchon* c'est de gauche, *le golf* c'est de droite, *les manifs* c'est de gauche. Pour savoir si un nouveau mot est de gauche ou de droite, l'IA va juste se baser sur son pré-entrainement qui a été fait sur une grosse partie d'internet, comme expliqué dans l'introduction.

Ci-dessous, l'ensemble final de mots utilisés pour le modèle le plus performant de GPT-3. Pour le modèle moins performant, j'utilise un plus gros ensemble de données.

In [3]:
training_set = [{"name":"Sarkozy", "out":"De droite"},
             {"name":"Hollande", "out":"De gauche"},
             {"name":"Macron", "out":"Les deux"}, # Certains ne vont pas être d'accord avec celle-ci...
             {"name":"La France", "out":"Les deux"},
             {"name":"L'argent", "out":"De droite"},
             {"name":"La bourse", "out":"De droite"},
             {"name":"Mélenchon", "out":"De gauche"},
             {"name":"Zemmour", "out":"De droite"},
             {"name":"Le sexe", "out":"Les deux"},
             {"name":"Les manifestations", "out":"De gauche"},
             {"name":"La politique", "out":"Les deux"},
             {"name":"Faire du golf", "out":"De droite"},
             {"name":"Le pastis", "out":"De gauche"}, # Certains m'ont dit qu'ils n'étaient pas d'accord avec ça
             {"name":"La sécurité sociale", "out":"De gauche"},
             {"name":"L'eau", "out":"Les deux"},
             {"name":"Les cigares", "out":"De droite"},
             {"name":"Le vélo", "out":"De gauche"},
             {"name":"Le bien", "out":"Les deux"}] # On tente de rendre l'IA plus ou moins neutre (spoiler: ça marche pas trop)

L'idée va être de rajouter une vingtième entrée à cette liste : le mot de l'utilisateur. Puis, on demande si ce mot est de gauche ou de droite.

**Le choix des mots**

Cet ensemble de mots est passé par beaucoup de versions différentes. Notamment, il est très important, sur un si petit jeu de données, de réfléchir à la proportion de mots de chaque catégorie. Par exemple, si je mets *"Les deux"* à tout ce qui n'est pas un politicien, alors je vais avoir beaucoup plus de *"Les deux"* quand je vais demander un nouveau mot, et le jeu sera moins drôle. 

**L'ordre des mots**

Autre chose importante : l'ordre des données d'entraînement. Ici, les exemples vont tous être à la suite dans une même requête (vous allez comprendre par la suite). Si on met tous les mots de gauche au début et tous les mots de droite à la fin, l'IA va très certainement répondre *"De droite"* pour quasiment tout, car le mot de l'utilisateur est ajouté à la fin. De même, si on alterne gauche, droite, gauche, droite et que l'on finit par droite (par exemple), l'IA risque de se concentrer sur le pattern au lieu de regarder le mot, et va automatiquement répondre que le mot suivant est de gauche.

Maintenant qu'on a notre liste de mots, il faut la transformer en un début de requête à compléter.

In [4]:
strFull = "De droite ou de gauche ?\n\n"

for x in training_set:
    strFull += "%s => %s\n"%(x["name"], x["out"])
    

On y est presque, il suffit d'ajouter une dernière ligne avec le mot entré par l'utilisateur, suivi du signe *"=>"*. Par exemple, si je veux savoir si les chaises sont de droite ou de gauche, je vais demander de compléter la requête suivante.

In [5]:
print(strFull+"Les chaises =>")

De droite ou de gauche ?

Sarkozy => De droite
Hollande => De gauche
Macron => Les deux
La France => Les deux
L'argent => De droite
La bourse => De droite
Mélenchon => De gauche
Zemmour => De droite
Le sexe => Les deux
Les manifestations => De gauche
La politique => Les deux
Faire du golf => De droite
Le pastis => De gauche
La sécurité sociale => De gauche
L'eau => Les deux
Les cigares => De droite
Le vélo => De gauche
Le bien => Les deux
Les chaises =>


On peut alors créer une fonction toute simple qui va envoyer des requêtes à OpenAI avec l'API *completion*. 

In [6]:
def degaucheoudedroite(prompt):
    g = openai.Completion.create(
          model="davinci",
          prompt=strFull+prompt+" =>",
          max_tokens=20,
          stop="\n",
          temperature=0.15
        )
    
    return g["choices"][0]["text"].strip()

J'y ai ajouté quelques paramètres:

* **model** est le nom du modèle que l'on veut utiliser pour compléter le texte. *Da vinci* est le meilleur, et quasiment le seul qui fonctionne bien pour notre tâche. *Curie* est le deuxième meilleur, très bon aussi, mais tout de même moins performant. Vous pouvez trouver la liste des modèles existant sur le site de OpenAI, et il en existe aussi pour des tâches précises. Il est aussi possible d'entraîner son propre modèle sur une tâche très précise à partir d'un modèle existant.

* **max_tokens** est le nombre maximum de tokens (1 token = un ou plusieurs caractères) que je veux recevoir en sortie. Puisque l'on veut seulement *"De gauche"*, *"De droite"* ou *"Les deux"*, je mets une petite valeur.

* **stop** est une séquence qui signifie que la complétion est terminée. Ici, un simple retour à la ligne.

* **temperature** est le paramètre le plus intéressant, et celui que j'ai le plus modifié. Plus il est haut, plus l'IA prendra des libertés dans sa réponse. S'il est bas, l'IA répondra quelque chose de peu original. *0.15* est une valeur assez basse, car je me suis dit que c'était mieux que l'IA retourne quelque chose de pas très original, mais qui soit cohérent.

Et ben voilà, c'est tout. On peut maintenant tester notre *"Intelligence artificielle"*  sur quelques exemples.

In [7]:
exemples = ["Trump", "Poutine", "Le champagne", "Le caviar", "Être vegan", "Le cassoulet", "CNews", "Le journal Libération",
           "Philippe Poutou", "La grève", "Les riches", "La manif pour tous", "Le parti socialiste", "La France Insoumise",
            "Les républicains", "La Suisse", "Le féminisme", "L'écologie", "Le cannabis", "La coke", "Le tennis", "La révolte",
           "Le tofu", "Un bon gros steak", "Un bon gros steak vegan", "Une manif", "Une manif d'identitaire"]

In [8]:
for exemple in exemples:
    result = degaucheoudedroite(exemple)
    print("%s : %s"%(exemple, result))

Trump : De droite
Poutine : De droite
Le champagne : De droite
Le caviar : De droite
Être vegan : De gauche
Le cassoulet : De gauche
CNews : De droite
Le journal Libération : De gauche
Philippe Poutou : De gauche
La grève : De gauche
Les riches : De droite
La manif pour tous : De droite
Le parti socialiste : De gauche
La France Insoumise : De gauche
Les républicains : De droite
La Suisse : De droite
Le féminisme : De gauche
L'écologie : De gauche
Le cannabis : De gauche
La coke : De droite
Le tennis : De droite
La révolte : De gauche
Le tofu : De gauche
Un bon gros steak : De droite
Un bon gros steak vegan : De gauche
Une manif : De gauche
Une manif d'identitaire : De droite


Plutôt pas mal !

Attention cependant, cette IA de OpenAI n'est pas gratuite d'utilisation. Les prix sont indiqués [sur cette page](https://openai.com/api/pricing/). Pour le modèle le plus performant, c'est 2 centimes pour 1000 tokens. Chaque requête fait environ 200 tokens (une autre raison qui explique pourquoi il y a peu d'exemples dans la requête), ce qui fait 1$ pour 250 requêtes. Ce n'était pas tenable pour moi, donc je suis passé à un autre modèle très rapidement (*curie*, qui est moins performant mais très bon tout de même).


Voilà, vous savez tout, et vous avez vu que ce n'est pas bien compliqué. Il reste juste à trouver une nouvelle idée qui fera le buzz ;)